# 데이터 전처리

마운트 폴더 지정

In [1]:
import sys

In [2]:
sys.path.append(r'C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET/src')

데이터 불러오기

In [3]:
import helper_functions
import pet.modeling
import pet.wrapper

In [4]:
import pandas as pd
import numpy as np

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import pet
import os
import numpy as np
import tqdm
import datasets
import pandas as pd

yelp_review_full(5중)

In [6]:
yelp_review_full_dataset = datasets.load_dataset("yelp_review_full")

2023-07-18 18:29:59,621 - WARNING - builder - Found cached dataset yelp_review_full (C:/Users/user/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

방법1

In [7]:
def get_yelp_review_full_data(show_progress_bars=False):

    if not show_progress_bars: datasets.logging.disable_progress_bar()
    
    # yelp_polarity_dataset = datasets.load_dataset('yelp_polarity')
    
    yelp_review_full_data = datasets.load_dataset('yelp_review_full')
    df_train = pd.DataFrame(yelp_review_full_dataset['train'][:2040])
    df_test = pd.DataFrame(yelp_review_full_dataset['test'])
    df_train['label'] = (df_train['label'] + 1).astype(str)
    df_test['label'] = (df_test['label'] + 1).astype(str)
    return df_train.loc[:40], df_train.loc[40:], df_test

방법2

In [7]:
def get_yelp_review_full_data_2(show_progress_bars=False):

    if not show_progress_bars: datasets.logging.disable_progress_bar()
    
    # yelp_polarity_dataset = datasets.load_dataset('yelp_polarity')
    
    yelp_review_full_data = datasets.load_dataset('yelp_review_full')
    df_train = pd.DataFrame(yelp_review_full_data['train'])
    df_test = pd.DataFrame(yelp_review_full_data['test'])
    df_train['label'] = (df_train['label'] + 1).astype(str)
    df_test['label'] = (df_test['label'] + 1).astype(str)
    return df_train, df_test

방법2로 진행

In [8]:
df_train, df_test = get_yelp_review_full_data_2(show_progress_bars=False)

2023-07-19 02:53:59,541 - WARNING - builder - Found cached dataset yelp_review_full (C:/Users/user/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


In [9]:
pd.value_counts(df_train["label"])

5    130000
2    130000
4    130000
1    130000
3    130000
Name: label, dtype: int64

In [10]:
pd.value_counts(df_test["label"])

1    10000
3    10000
2    10000
4    10000
5    10000
Name: label, dtype: int64

문장 길이 제한

In [11]:
df_train_400 = df_train[df_train['text'].str.len().between(400, 600)].reset_index(drop=True)

In [12]:
df_train_400

,label,text
0,5,dr. goldberg offers everything i look for in a...
1,4,Been going to Dr. Goldberg for over 10 years. ...
2,3,Decent range somewhat close to the city. The ...
3,1,This place is absolute garbage... Half of the...
4,5,Let there be no question: Alexions owns the be...
...,...,...
115074,3,Honestly i expected much much more out of a la...
115075,5,"As apart of our daily business, we value the w..."
115076,2,A man came to the house to estimate. He gave m...
115077,5,MACARONS!!!! I've died and gone to heaven. \n\...


In [13]:
pd.value_counts(df_train_400["label"])

3    23776
4    23232
5    23029
2    22894
1    22148
Name: label, dtype: int64

train data

In [14]:
def sample_data_by_label(dataframe, label_column, label_values, num_samples, random_state=42):
    sampled_data = pd.concat([dataframe[dataframe[label_column] == label].sample(n=num_samples, random_state=random_state)
                             for label in label_values], ignore_index=True)
    return sampled_data

In [17]:
df_train_400_train = sample_data_by_label(df_train_400, 'label', ['1', '2', '3', '4', '5'], 20, random_state=42)

In [18]:
df_train_400_train

,label,text
0,1,Stopped here for a byte to eat with about 8 pe...
1,1,Yuk-O. I work just down the street and read th...
2,1,So disappointed. The BWW that we normally go t...
3,1,I wanted put zero but yelp does not allow.Wors...
4,1,Don't waste your time in vegas. \n\nThe good: ...
...,...,...
95,5,We are so thrilled with the service and window...
96,5,"Good, healthy choices of typical Mediterranean..."
97,5,I haven't even gotten my food yet and this pla...
98,5,Just ate here today and this place is great! T...


In [19]:
df_train_400_train.to_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET/data/df_train_400_train.csv", index = False)

In [20]:
# unlabeled data  = df_train_400 - df_train_400_train

def merge_and_filter(dataframe1, dataframe2, on_columns, how='left'):
    
    merged_df = dataframe1.merge(dataframe2, on=on_columns, how=how, indicator=True)
    filtered_df = merged_df[merged_df['_merge'] == 'left_only']
    filtered_df = filtered_df.drop(columns=['_merge']).reset_index(drop=True)

    return filtered_df

In [21]:
df_train_400_train_unlabeled = merge_and_filter(df_train_400, df_train_400_train, ['label', 'text'], how='left')

In [27]:
type(df_train_400_train_unlabeled["text"][0])

str

In [23]:
df_train_400_train_unlabeled.to_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET/data/df_train_400_train_unlabeled.csv", index = False)

test data

In [24]:
df_test.to_csv("C:/Users/user/Desktop/논문리뷰/Pattern-Exploiting-Training(PET)/code/testPET/data/df_test.csv", index = False)